In [1]:
import geopandas as gpd
# import pandas as pd
# import matplotlib.pyplot as plt
# from owslib.wfs import WebFeatureService
# from owslib.util import ResponseWrapper
# from shapely.geometry import box, Polygon
# import requests
# from os import path

In [2]:
# --- KONFIGURACJA ŚCIEŻKI ---
gmina_shp_path = 'GMN.shp'
gmina_nazwa_szukana = 'Wińsko'

print(f"Próba wczytania pliku granic gmin z: {gmina_shp_path}")

try:
    gdf_teryt = gpd.read_file(gmina_shp_path, encoding='utf-8')
    print(f"Wczytano {len(gdf_teryt)} obiektów gmin.")

    # Ujednolicenie CRS na EPSG 2180
    if gdf_teryt.crs != 'EPSG:2180':
        gdf_teryt = gdf_teryt.to_crs('EPSG:2180')
        print(f"Reprojekcja do EPSG:2180 zakończona.")

    # Znalezienie odpowiednich kolumn dla nazwy i typu
    kolumna_gmin = next((col for col in gdf_teryt.columns if 'NAZWA' in col.upper()), None)
    
    if kolumna_gmin:
        gdf_gmina = gdf_teryt[
            gdf_teryt[kolumna_gmin].notna() &
            (gdf_teryt[kolumna_gmin] == gmina_nazwa_szukana)
            ]
        if not gdf_gmina.empty:
            gmina_maska = gdf_gmina.iloc[0].geometry
            minx, miny, maxx, maxy = gdf_gmina.total_bounds
            bbox = (minx, miny, maxx, maxy)
            print(f"✅ Znaleziono geometrię gminy {gmina_nazwa_szukana} (CRS: {gdf_gmina.crs}).")
            print(f"Wyznaczony BBOX (EPSG:2180): {bbox}")
        else:
            raise ValueError(f"❌ Nie znaleziono gminy '{gmina_nazwa_szukana}'. Sprawdź pisownię i kolumnę '{kolumna_gmin}'.")
    else:
        raise ValueError("❌ Nie znaleziono kolumny zawierającej nazwę gminy. Sprawdź `gdf_teryt.head()`.")
except FileNotFoundError:
    print(f"❌ Błąd: Plik **{gmina_shp_path}** nie został znaleziony! Upewnij się, że pobrałeś plik SHP z granicami gmin (PRG) i umieściłeś go w odpowiednim miejscu.")
except Exception as e:
    print(f"❌ Wystąpił błąd ogólny podczas przetwarzania TERYT: {e}")

Próba wczytania pliku granic gmin z: GMN.shp
Wczytano 1 obiektów gmin.
✅ Znaleziono geometrię gminy Wińsko (CRS: EPSG:2180).
Wyznaczony BBOX (EPSG:2180): (np.float64(321977.94), np.float64(392371.47), np.float64(347015.71), np.float64(408400.12))


In [3]:
# --- Konfiguracja ---
bdot_file_path = './dane_BDOT/PL.PZGiK.337.0222__OT_BUBD_A.xml'
target_crs = 'EPSG:2180'
target_layer_type = 'Budynek'

print(f"\n--- Wczytywanie danych BDOT10k z pliku: {bdot_file_path} ---")
try:
    # 1. Wczytanie pliku GML
    gdf_powiat_raw = gpd.read_file(bdot_file_path)
    print(f"Wczytano {len(gdf_powiat_raw)} obiektów z pliku powiatowego.")

    # 2. Ujednolicenie CRS
    if gdf_powiat_raw.crs != target_crs:
        gdf_powiat_raw = gdf_powiat_raw.to_crs(target_crs)
        print("Reprojekcja danych BDOT10k zakończona.")

    # 3. Iloczynowa algebra mapowa do geometrii maski gminy Wińsko
    gdf_maska = gpd.GeoDataFrame(index=[0], crs=target_crs, geometry=[gmina_maska])

    # Użycie sjoin z opcją 'intersects' (przecięcie)
    gdf_iloczyn = gpd.sjoin(gdf_powiat_raw, gdf_maska, how="inner", predicate='intersects')

    # Usunięcie zbędnych kolumn z sjoin (index_right, ewentualnie inne)
    gdf_iloczyn.drop(columns=['index_right'], inplace=True, errors='ignore')

    print(f"✅ Po wycięciu w granicach gminy {gmina_nazwa_szukana.capitalize()} pozostało {len(gdf_iloczyn)} obiektów.")

    # Sprawdzenie, czy dane zostały pomyślnie wczytane i wycięte
    if gdf_iloczyn is not None and not gdf_iloczyn.empty:
        # Harmonizacja schematu
        target_columns = ['id_zrodla', 'typ_obiektu', 'geometry']
    
        gdf_harmonized = gpd.GeoDataFrame(columns=target_columns, crs=target_crs)
    
        if 'id' in gdf_iloczyn.columns:
            gdf_harmonized['id_zrodla'] = gdf_iloczyn['id']
        elif 'gml_id' in gdf_iloczyn.columns:
            gdf_harmonized['id_zrodla'] = gdf_iloczyn['gml_id']
        else:
            gdf_harmonized['id_zrodla'] = gdf_iloczyn.index.astype(str)
    
        gdf_harmonized['typ_obiektu'] = target_layer_type
        gdf_harmonized['geometry'] = gdf_iloczyn['geometry']
    
        print(f"\n--- Zharmonizowana GeoDataFrame ({target_layer_type}) ---")
        print(gdf_harmonized.head(2))
    
        # Zapis
        output_filename = 'bdot10k_' + target_layer_type.lower() + '_harmonized.gpkg'
        gdf_harmonized.to_file(r'./dane_Harmonizacja/' + output_filename, driver='GPKG')
        print(f"✅ Zharmonizowane dane zapisane do pliku: {output_filename}")
    
    elif gdf_iloczyn is not None and gdf_iloczyn.empty:
        print("\n⚠️ Brak obiektów BDOT10k w pliku powiatowym, które by przecinały geometrię gminy Wińsko.")
except FileNotFoundError:
    print(f"❌ Błąd: Plik **{bdot_file_path}** nie został znaleziony! Upewnij się, że pobrałeś paczkę i zmieniłeś ścieżkę.")
    gdf_iloczyn = None
except Exception as e:
    print(f"❌ Wystąpił błąd podczas wczytywania/przetwarzania: {e}")
    gdf_iloczyn = None



Skipping field funkcjaSzczegolowaBudynku: unsupported OGR type: 5



--- Wczytywanie danych BDOT10k z pliku: ./dane_BDOT/PL.PZGiK.337.0222__OT_BUBD_A.xml ---
Wczytano 22328 obiektów z pliku powiatowego.
✅ Po wycięciu w granicach gminy Wińsko pozostało 6468 obiektów.

--- Zharmonizowana GeoDataFrame (Budynek) ---
                                  id_zrodla typ_obiektu  \
15  id_1397F7C2-FF04-41C5-E053-CA2BA8C0BF77     Budynek   
16  id_1397F7C2-F51D-41C5-E053-CA2BA8C0BF77     Budynek   

                                             geometry  
15  POLYGON ((332039.61 396444.67, 332041.65 39645...  
16  POLYGON ((330536.84 406765.74, 330537.08 40677...  
✅ Zharmonizowane dane zapisane do pliku: bdot10k_budynek_harmonized.gpkg
